## Transverse Ising Model using VQE


## Define Ising Model Hamiltonian: $H_I$

In [5]:
from typing import List, Tuple, Any
from pytket.utils import QubitPauliOperator
from pytket.pauli import QubitPauliString, Pauli
from pytket import Qubit

def transverse_ising_model_hamiltonian(Ising_Terms: List[Tuple[int, int, int, int]]) -> QubitPauliOperator:
    qpo_dict = {}
    for term in Ising_Terms:
        term_string = QubitPauliString([Qubit(term[0]), Qubit(term[1])], [Pauli.Z, Pauli.Z])
        qpo_dict[term_string] = term[2]
        term_string2 = QubitPauliString([Qubit(term[0])], [Pauli.X])
        qpo_dict[term_string2] = term[3]
    return QubitPauliOperator(qpo_dict)

sample_model = [(0,1,4.2,3.2), (1,0, 2.3, 5.6)]
num_spins = 2
Ising_Ham = transverse_ising_model_hamiltonian(sample_model)
print(Ising_Ham)

{(Zq[0], Zq[1]): 2.30000000000000, (Xq[0]): 3.20000000000000, (Xq[1]): 5.60000000000000}


## Hamiltonian Circuit

In [6]:
from pytket.utils import gen_term_sequence_circuit
from pytket import Circuit
from pytket.circuit import display

hamiltonian_circuit = gen_term_sequence_circuit(Ising_Ham, Circuit(num_spins))
display.render_circuit_jupyter(hamiltonian_circuit)

In [7]:
from pytket.transform import Transform

Transform.DecomposeBoxes().apply(hamiltonian_circuit)
display.render_circuit_jupyter(hamiltonian_circuit)

## Define the Variational Ansatz

In [48]:
def hea(params):
    ansatz = Circuit(2)
    for i in range(2):
        ansatz.Ry(params[i], i)
    ansatz.Rx(math.pi/2, 0)
    ansatz.Ry(-math.pi/2, 1)
    for i in range(1):
        ansatz.CX(i, i + 1)
    ansatz.Rz(params[2], 1)
    for i in range(1):
        ansatz.CX(i, i + 1)
    ansatz.Rx(-math.pi/2, 0)
    ansatz.Ry(math.pi/2, 1)
    return ansatz

## Construct VQE Circuit

Now lets define a function to create our entire QAOA circuit. For $p$ QAOA layers we expect that our circuit will require $2p$ parameters. Here we will pass and cost mixer parameters in as a list where the length of the list defines the number of layers.

In [59]:
def ising_circuit(t_model: List[Tuple[int, int, int, int]],
                         n_spins: int,
                         params: List[float]) -> Circuit:
    
    # initial state
    i_circuit = hea(params)
    i_circuit.append(gen_term_sequence_circuit(Ising_Ham, Circuit(n_spins)))
        
    Transform.DecomposeBoxes().apply(i_circuit)
    return i_circuit

We also need to extract our energy expectation values from a `BackendResult` object after our circuit is processed by the device/simulator. We do this with the `get_max_cut_energy` function below. Note that the fact that the maxcut Hamiltonian contains only commuting terms means that we do not need to calculate our energy expectation using multiple measurement circuits. This may not the the case for a different problem Hamiltonian.

In [65]:
from pytket.backends.backend import Backend
from pytket.utils import get_operator_expectation_value
from pytket.partition import PauliPartitionStrat
from typing import Callable
import numpy as np

def Ising_instance(
    backend: Backend,
    compiler_pass: Callable[[Circuit], bool],
    guess_params: np.array,
    t_model = [(0,1,2.5,4.5), (1,0,3.5,2.5)],
    n_spins = 2,
    seed: int = 12345,
    shots: int = 5000,
) -> float:
    # step 1: get state guess
    
    my_prep_circuit = ising_circuit(t_model, n_spins, guess_params)
    return get_operator_expectation_value(
            my_prep_circuit,
            Ising_Ham,
            backend,
            n_shots=4000,
            partition_strat=PauliPartitionStrat.CommutingSets,
        ).real

## Optimise Energy by Guessing Parameters

In [66]:
def qaoa_optimise_energy(compiler_pass: Callable[[Circuit], bool],
                         backend: Backend,
                         iterations: int = 100,
                         n: int = 3,
                         shots: int = 5000,
                         seed: int= 12345):
    
    highest_energy = 0    
    params = [0 for i in range(n)]    
    rng = np.random.default_rng(seed)
    # guess some angles (iterations)-times and try if they are better than the best angles found before
    
    for i in range(iterations):
        
        guess_params = rng.uniform(0, 1, n)
        
        qaoa_energy = Ising_instance(backend,
                                    compiler_pass,
                                    guess_params,
                                    seed=seed,
                                    shots=shots)
        
        if(qaoa_energy > highest_energy):
            
            print("new highest energy found: ", qaoa_energy)
            
            best_guess_params = np.round(guess_params, 3)
            highest_energy = qaoa_energy
            
    print("highest energy: ", highest_energy)
    print("best guess parameters: ", best_guess_params)
    return best_guess_params

## Calculate the State for the final Parameters

In [70]:
from pytket.backends.backendresult import BackendResult

def qaoa_calculate(backend: Backend,
                   compiler_pass: Callable[[Circuit], bool],
                   shots: int = 5000,
                   iterations: int = 100,
                   seed: int = 12345,
                  ) -> BackendResult:
    
    # find the parameters for the highest energy
    best_params = qaoa_optimise_energy(compiler_pass,
                                                 backend,
                                                 iterations,
                                                 3,
                                                 shots=shots,
                                                 seed=seed)
    
    # get the circuit with the final parameters of the optimisation:
    t_model = [(0,1,2.5,4.5), (1,0,3.5,2.5)]
    my_qaoa_circuit = ising_circuit(t_model, 2, best_params)

    my_qaoa_circuit.measure_all()

    compiler_pass(my_qaoa_circuit)
    handle = backend.process_circuit(my_qaoa_circuit, shots, seed=seed)

    result = backend.get_result(handle)    
    
    return result

## Results with the Noiseless Simulator

In [71]:
from pytket.extensions.qiskit import AerBackend
import math
backend = AerBackend()
comp = backend.get_compiled_circuit

In [72]:
%%time
res = qaoa_calculate(backend, backend.default_compilation_pass(2).apply, shots = 5000, iterations = 100, seed=12345)

new highest energy found:  0.30969999999999953
new highest energy found:  3.43395
new highest energy found:  3.4583999999999997
new highest energy found:  3.64505
new highest energy found:  5.2217
new highest energy found:  5.87115
new highest energy found:  6.127949999999999
highest energy:  6.127949999999999
best guess parameters:  [0.031 0.64  0.653]
CPU times: user 11.5 s, sys: 1.45 s, total: 12.9 s
Wall time: 13.5 s
